In [1]:
import pandas as pd
import numpy as np

In [2]:
imports = pd.read_csv('cp.csv')

In [3]:
#las borro para quitar columnas visualmente.
imports.drop(columns=['tipoI','tipoII'],inplace=True)

In [4]:
ruta_medicamentos = 'medicamentosHomologado.csv'
medicines = pd.read_csv(ruta_medicamentos)

In [5]:
#creacion de columnas para ir detectando cambios y más
imports['cnis'] = ''
imports['status_medicamento'] = ''
imports['empresa'] = ''
imports['unidosis'] = ''
imports['c_r_c_llenado_datos'] = ''

In [ ]:
#retorna info relevante de medicamentos segun su NOMBRE y MARCA
medicines[['CONCENTRACION','UNIDAD CONCENTRACION','PRESENTACION','FORMA FARMA']][ (medicines['MEDICAMENTO'] == 'ruxolitinib' ) & (medicines['NOMBRE COMERCIAL']=='jakavi') ]

In [ ]:
#retorna info relevante de medicamentos segun su NOMBRE 
medicines[['CONCENTRACION','UNIDAD CONCENTRACION','FORMA FARMA','MEDICAMENTO','NOMBRE COMERCIAL']][ medicines['MEDICAMENTO'] == 'everolimus' ]

In [6]:
#es necesario porque concentracion era float64 y le agregaba .0 a los numeros enteros y un 0.5 lo hacia 0
imports['concentracion'] =  imports['concentracion'].astype(str).str.replace(".0","",regex=False)

## funciones

In [7]:
def match_imports_medicine(imp_row: pd.Series) -> list:
    """ 
    receive a row of imports, will try to do a match in base of 
    nombre,marca and presentation of medicine in cuestion between 
    imports and medicines dataframes."""
    #checamos si hace match perfecto(nombre,marca,presentacion etc)
    primer_intento =  medicines[(medicines['MEDICAMENTO']==imp_row['nombre']) & 
                                (medicines['NOMBRE COMERCIAL']==imp_row['marca']) &
                                (medicines['CONCENTRACION']==imp_row['concentracion']) & 
                                (medicines['UNIDAD CONCENTRACION']==imp_row['unidad_concentracion']) & 
                                (medicines['PRESENTACION']==imp_row['presentacion']) & 
                                (medicines['FORMA FARMA']==imp_row['forma_farma']) ].to_dict('records')
    if len(primer_intento) > 0:
        #print(f"match perfecto {len(primer_intento)}")
        return (imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],
                primer_intento[0]['EMPRESA'], primer_intento[0]['CONCENTRACION'], primer_intento[0]['UNIDAD CONCENTRACION'],
                primer_intento[0]['PRESENTACION'], primer_intento[0]['FORMA FARMA'], primer_intento[0]['CNIS'], primer_intento[0]['STATUS'],'perfect match')
    
    #si no hizo match perfecto hay que ver que le falta y cuantos match con el mayor numero de coincidencias hace
    df_dict = medicines[ (medicines['MEDICAMENTO']==imp_row['nombre']) & (medicines['NOMBRE COMERCIAL']==imp_row['marca']) ].to_dict('records')
    if len(df_dict) == 0:
        print(f"sin match para{imp_row['nombre']}, {imp_row['marca']}")
        return (imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,'sin match')
    
    matches = 0 # número de matches entre imports y medicamentos con el mismo numero de coincidencias
    coincidencias = 0 #coincidencia entre imports y medicamentos en sus columnas, concentracion,unidad_concentracion, presentacion, forma farma
    medicine_match = '' #
    for medic in df_dict:
        contador = 0
        if medic['CONCENTRACION'] == imp_row['concentracion']:
            contador +=1 
        if medic['UNIDAD CONCENTRACION'] == imp_row['unidad_concentracion']:
            contador +=1
        if medic['PRESENTACION'] == imp_row['presentacion']:
            contador +=1
        if medic['FORMA FARMA'] == imp_row['forma_farma']:
            contador += 1

        #si hay coincidencias y es mayor en numero de estas que el match anterior(en caso de que halla)
        if contador > coincidencias:
            coincidencias = contador
            matches = 1     
            medicine_match =str(medic['PRESENTACION']) +  medic['FORMA FARMA'] + str(medic['CONCENTRACION']) + medic['UNIDAD CONCENTRACION']
        #si existen las mismas coincidencias que el match anterior 
        elif contador > 0 and contador == coincidencias:
            matches += 1    
            medicine_match += " || " + str(medic['PRESENTACION']) +  medic['FORMA FARMA'] + str(medic['CONCENTRACION']) + medic['UNIDAD CONCENTRACION']

    c_r_c = f"{coincidencias} coincidencias en {matches} matches: {medicine_match}"
    return (imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
            imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,c_r_c)

## match

In [9]:
imports[imports['nombre_y_marca']=='NM'].apply(match_imports_medicine, axis=1, result_type='expand').to_csv('final_match.csv')

In [ ]:
#prueba para buscar el match perfecto
medicines[(medicines['MEDICAMENTO']=='dabrafenib') & 
        (medicines['NOMBRE COMERCIAL']=='tafinlar') &
        (medicines['CONCENTRACION']=='75') & 
        (medicines['UNIDAD CONCENTRACION']=='mg') & 
        (medicines['PRESENTACION']=='28') & 
        (medicines['FORMA FARMA']=='capsulas') ]
                                

### match perfecto (para probar)

In [34]:
def perfect_match_imports_medicine(imp_row: pd.Series) -> list:
    """ para generar la primera version de match perfectos para prueba"""
    #checamos si hace match perfecto(nombre,marca,presentacion etc)
    primer_intento =  medicines[(medicines['MEDICAMENTO']==imp_row['nombre']) & 
                                (medicines['NOMBRE COMERCIAL']==imp_row['marca']) &
                                (medicines['CONCENTRACION']==imp_row['concentracion']) & 
                                (medicines['UNIDAD CONCENTRACION']==imp_row['unidad_concentracion']) & 
                                (medicines['PRESENTACION']==imp_row['presentacion']) & 
                                (medicines['FORMA FARMA']==imp_row['forma_farma']) ].to_dict('records')
    if len(primer_intento) > 0:
        return (primer_intento[0]['EMPRESA'],  primer_intento[0]['CNIS'], primer_intento[0]['STATUS'],primer_intento[0]['UNIDOSIS'], 'perfect match')
    else:
        return (None,None,None,None,'sin match perfecto')

In [31]:
complete_imports = pd.read_csv('cp_complete.csv')

In [32]:
complete_imports['concentracion'] =  complete_imports['concentracion'].astype(str).str.replace(".0","",regex=False)

In [35]:
complete_imports[['empresa','cnis','status_medicamento', 'unidosis','c_r_c_llenado_datos']] = complete_imports[complete_imports['nombre_y_marca']=='NM'].apply(perfect_match_imports_medicine, axis=1, result_type='expand').copy()

In [36]:
complete_imports[complete_imports['c_r_c_llenado_datos']=='perfect match'].to_csv('perfect_match_v1.csv')